In [ ]:
# %matplotlib inline

import numpy as np
import imp
import os
from numba import njit
import sys
import h5py

FILE_NAME = os.path.abspath('')
FILE_PATH = os.path.dirname(FILE_NAME)
FILE_BASE = os.path.basename(FILE_NAME)

import fastpli.simulation
import fastpli.analysis
import fastpli.tools
import fastpli.io
imp.reload(fastpli)

import matplotlib.pyplot as plt
import tikzplotlib
from skimage.external import tifffile as tif

import vector_field_generation as vfg
imp.reload(vfg)

np.random.seed(42)

print(fastpli.__version__)

In [ ]:
scale = 2
voxel_size = 0.1

In [ ]:
simpli = fastpli.simulation.Simpli()
simpli.omp_num_threads = 2
print(FILE_PATH)
simpli.fiber_bundles = fastpli.io.fiber_bundles.load(os.path.join(FILE_NAME, '..', 'data', 'models', 'cube_2pop_psi_0.5_omega_0.0_.solved.h5'))
# simpli.fiber_bundles_properties = [[(0.75, 0, 0, 'b'),
#                                     (1.0, dn, 0, model)]] * len(
#                                         simpli.fiber_bundles)

simpli.voxel_size = voxel_size  # in µm meter
simpli.set_voi([-5] * 3, [5] * 3)  # in µm meter

voxel_size_0 = simpli.voxel_size

# fig, axs = plt.subplots(2, 3)
for m, (dn, model) in enumerate([(-0.002, 'p'), (0.004, 'r')]):
    simpli.fiber_bundles_properties = [[(0.75, 0, 0, 'b'),
                                        (1.0, dn, 0, model)]] * len(
                                        simpli.fiber_bundles)

    # low resolution   
    simpli.voxel_size = voxel_size_0  # in µm meter
    tissue, optical_axis, tissue_properties = simpli.generate_tissue()

    # high resolution
    simpli.voxel_size = voxel_size_0 / scale  # in µm meter
    print(f"scale: {scale}, model: {model} -> {simpli.memory_usage('MB'):.0f} MB")
    if simpli.memory_usage('MB') > 128**12:
        print("MEMORY!")
        sys.exit(1)
    tissue_high, optical_axis_high, tissue_properties = simpli.generate_tissue(
    )
    print("IntpVecField")
    vf_intp = vfg.IntpVecField(tissue, optical_axis, scale, True)
    print("diff")
    vf_diff = np.linalg.norm(vfg.VectorOrientationSubstractionField(
        optical_axis_high, vf_intp),
                             axis=-1)
    
    vmax = np.amax(vf_diff)
    print(f"vmax: {vmax}")
#     axs[m, 0].imshow(vf_diff[vf_diff.shape[0] // 2, :, :],
#                      vmin=0,
#                      vmax=vmax)
#     axs[m, 1].imshow(vf_diff[:, vf_diff.shape[1] // 2, :],
#                      vmin=0,
#                      vmax=vmax)
#     pcm = axs[m, 2].imshow(vf_diff[:, :, vf_diff.shape[2] // 2],
#                            vmin=0,
#                            vmax=vmax)

    
    tmp = np.logical_or(np.logical_and(tissue_high>0, tissue_high%2==0), vf_diff != 0)
    tmp = vf_diff[tmp].flatten()
    val = np.mean(tmp)
    val_std = np.std(tmp)
    val_min = np.min(tmp)
    val_max = np.max(tmp)
    val_med = np.median(tmp)
    val_qnt = np.quantile(tmp, [0.25, 0.5, 0.75])
    
    print(f"val: {val} \pm {val_std}: {val_med}, {val_qnt}, {val_min}, {val_max}")
    
#     x = np.linspace(0, 1, 1000, True)
#     plt.plot(x, np.quantile(tmp,x))

    tmp = np.sort(tmp)
    x = np.arange(tmp.size)
    plt.plot(x/tmp.size, tmp)
    
#     if m == 0:
#         fig.colorbar(pcm, ax=axs[:, 2])

    # print(tissue_high.shape)
    # np.savez(f"test_vfdiff_{scale}.npz")
    # tif.imsave(f"vfdiff_{scale}.tiff", vf_diff, bigtiff=True)
    # tif.imsave(f"label_{scale}.tiff", tissue_high, bigtiff=True)

# plt.show()
# tikzplotlib.save(f"test_{scale}.tex")

# plt.show()

In [ ]:
phi = np.linspace(-0.1*np.pi, 1.1*np.pi, 100, True)
theta = phi*0
v = np.array(np.array([np.sin(phi)*np.cos(theta), np.cos(phi)*np.cos(theta), np.sin(theta)]).T)
v0 = np.zeros_like(v)
v0[:,0] = 1
vf_diff = np.linalg.norm(vfg.VectorOrientationSubstractionField(v0, v), axis=-1)

plt.plot(phi/np.pi*180, vf_diff)
print("WISO > 1 ??")